In [1]:
import pandas as pd
import re

In [2]:
data=pd.read_csv("Tweets.csv",nrows=50)

In [3]:
data.head()
data.shape
data.dtypes
data["text"].astype(str)

0                   @VirginAmerica What @dhepburn said.
1     @VirginAmerica plus you've added commercials t...
2     @VirginAmerica I didn't today... Must mean I n...
3     @VirginAmerica it's really aggressive to blast...
4     @VirginAmerica and it's a really big bad thing...
5     @VirginAmerica seriously would pay $30 a fligh...
6     @VirginAmerica yes, nearly every time I fly VX...
7     @VirginAmerica Really missed a prime opportuni...
8       @virginamerica Well, I didn't…but NOW I DO! :-D
9     @VirginAmerica it was amazing, and arrived an ...
10    @VirginAmerica did you know that suicide is th...
11    @VirginAmerica I &lt;3 pretty graphics. so muc...
12    @VirginAmerica This is such a great deal! Alre...
13    @VirginAmerica @virginmedia I'm flying your #f...
14                               @VirginAmerica Thanks!
15        @VirginAmerica SFO-PDX schedule is still MIA.
16    @VirginAmerica So excited for my first cross c...
17    @VirginAmerica  I flew from NYC to SFO las

In [4]:
#lets check percentage of missing values 
data.isnull().sum()*100/len(data)

tweet_id                          0.0
airline_sentiment                 0.0
airline_sentiment_confidence      0.0
negativereason                   70.0
negativereason_confidence        58.0
airline                           0.0
airline_sentiment_gold          100.0
name                              0.0
negativereason_gold             100.0
retweet_count                     0.0
text                              0.0
tweet_coord                      88.0
tweet_created                     0.0
tweet_location                   30.0
user_timezone                    26.0
dtype: float64

In [5]:
#very big percentage for   airline_sentiment_gold,negativereason_gold,tweet_coord lets drop them

data.drop(['airline_sentiment_gold','negativereason_gold','tweet_coord'],axis=1,inplace=True)

In [6]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [7]:
#fillna with most frequent values 
data["negativereason"].fillna(data['negativereason'].mode().iloc[0],inplace=True)

data["negativereason_confidence"].fillna(data['negativereason_confidence'].mode().iloc[0],inplace=True)
data["tweet_location"].fillna(data["tweet_location"].mode().iloc[0],inplace=True)
data["user_timezone"].fillna(data["user_timezone"].mode().iloc[0],inplace=True)

In [8]:
#check for missing values 
data.isnull().sum()

tweet_id                        0
airline_sentiment               0
airline_sentiment_confidence    0
negativereason                  0
negativereason_confidence       0
airline                         0
name                            0
retweet_count                   0
text                            0
tweet_created                   0
tweet_location                  0
user_timezone                   0
dtype: int64

In [9]:

#perfect no missing values 

In [10]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,Customer Service Issue,1.0000,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,Los Angeles,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,Customer Service Issue,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,Los Angeles,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,Customer Service Issue,1.0000,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,Los Angeles,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,Los Angeles,Pacific Time (US & Canada)


In [11]:
#lets check for duplicate rows 
data.duplicated().sum()

0

In [12]:
#we have 39 duplicated rows lets drop them 
data.drop_duplicates(inplace=True)

In [13]:
#perfect
data.duplicated().sum()

0

In [14]:
#lets start the preporcessing of our text data 

In [15]:
#convert to lowercase
data["text"]=data["text"].str.lower()
data["negativereason"]=data["negativereason"].str.lower()
data["airline"]=data["airline"].str.lower()
data["tweet_location"]=data["tweet_location"].str.lower()
data["user_timezone"]=data["user_timezone"].str.lower()

In [16]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,customer service issue,1.0000,virgin america,cairdin,0,@virginamerica what @dhepburn said.,2015-02-24 11:35:52 -0800,los angeles,eastern time (us & canada)
1,570301130888122368,positive,0.3486,customer service issue,0.0000,virgin america,jnardino,0,@virginamerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,los angeles,pacific time (us & canada)
2,570301083672813571,neutral,0.6837,customer service issue,1.0000,virgin america,yvonnalynn,0,@virginamerica i didn't today... must mean i n...,2015-02-24 11:15:48 -0800,lets play,central time (us & canada)
3,570301031407624196,negative,1.0000,bad flight,0.7033,virgin america,jnardino,0,@virginamerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,los angeles,pacific time (us & canada)
4,570300817074462722,negative,1.0000,can't tell,1.0000,virgin america,jnardino,0,@virginamerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,los angeles,pacific time (us & canada)


In [17]:
data["text"][21]

'@virginamerica i love this graphic. http://t.co/ut5grrwaaa'

In [18]:
#lets remove urls and emojis 
#define regex pattern
#url_pattern=re.compile(r'https?://\s+')
#define function to remove urls from text
def remove_urls(text):
    url_pattern = re.compile('https?://\S+|www\.\S+')
    
    return url_pattern.sub("",text)

#apply function to column "text"
data["text"]=data["text"].apply(remove_urls)



In [19]:
data["text"][21]

'@virginamerica i love this graphic. '

In [20]:
data["text"][18]

'i ❤️ flying @virginamerica. ☺️👍'

In [21]:
emoji_dict = {
    "😀": "grinning face",
    "😃": "grinning face with big eyes",
    "😄": "grinning face with smiling eyes",
    "😁": "beaming face with smiling eyes",
    "😆": "grinning squinting face",
    "😅": "grinning face with sweat",
    "😂": "face with tears of joy",
    "🤣": "rolling on the floor laughing",
    "😊": "smiling face with smiling eyes",
    "😇": "smiling face with halo",
    "🙂": "slightly smiling face",
    "🙃": "upside-down face",
    "😉": "winking face",
    "😌": "relieved face",
    "😍": "smiling face with heart-eyes",
    "😘": "face blowing a kiss",
    "😗": "kissing face",
    "😙": "kissing face with smiling eyes",
    "😚": "kissing face with closed eyes",
    "😋": "face savoring food",
    "😛": "face with tongue",
    "😜": "winking face with tongue",
    "😝": "squinting face with tongue",
    "🤑": "money-mouth face",
    "🤗": "hugging face",
    "🤓": "nerd face",
    "😎": "smiling face with sunglasses",
    "🤡": "clown face",
    "🤠": "cowboy hat face",
    "😏": "smirking face",
    "😒": "unamused face",
    "😞": "disappointed face",
    "😔": "pensive face",
    "😟": "worried face",
    "😕": "confused face",
    "🙁": "slightly frowning face",
    "☹️": "frowning face",
    "😣": "persevering face",
    "😖": "confounded face",
    "😫": "tired face",
    "😩": "weary face",
    "😤": "face with steam from nose",
    "😠": "angry face",
    "😡": "pouting face",
    "😶": "face without mouth",
    "😐": "neutral face",
    "😑": "expressionless face",
    "😯": "hushed face",
    "😦": "frowning face with open mouth",
    "😧": "anguished face",
    "😮": "face with open mouth",
    "😲": "astonished face",
    "😵": "dizzy face",
    "😳": "flushed face",
    "😱": "face screaming in fear",
    "😨": "fearful face",
    "😰": "anxious face with sweat",
    "😢": "crying face",
    "😥": "sad but relieved face",
    "🤤": "drooling face",
    "😭": "loudly crying face",
    "😓": "downcast face with sweat",
    "😪": "sleepy face",
    "😴": "sleeping face",
    "🙄": "face with rolling eyes",
    "🤔": "thinking face",
    "🤥": "lying face",
    "😬": "grimacing face",
    "🤐": "zipper-mouth face",
    "🤢": "nauseated face",
    "🤧": "sneezing face",
    "😷": "face with medical mask",
    "🤒": "face with thermometer",
    "🤕": "face with head-bandage",
    "🤢": "nauseated face",
    "🤮": "face vomiting",
    "🤠": "cowboy hat face",
    "🤡": "clown face",
    "👹": "ogre",
    "👺": "goblin",
    "💩": "pile of poo",
    "👻": "ghost",
    "💀": "skull",
    "☠️": "skull and crossbones",
    "👽": "alien",
    "👾": "alien monster",
    "🤖": "robot",
    "🎃": "jack-o-lantern",
    "😺": "grinning cat face",
    "😸": "grinning cat face with smiling eyes",
    "😹": "cat face with tears of joy",
    "👍": "thumps up",
    "❤️": "love"}
def replace_emojis(text):
    # Create a regular expression pattern to match emojis
    emoji_pattern = re.compile("|".join(re.escape(emoji) for emoji in emoji_dict.keys()))
    
    # Replace emojis with their text representations
    return emoji_pattern.sub(lambda x: emoji_dict[x.group()], text)

# Apply the function to the 'text' column
data['text'] = data['text'].apply(replace_emojis)


In [22]:
data["text"][18]

'i love flying @virginamerica. ☺️thumps up'

In [23]:
## import re
#lets remove special characters etc
data["text"] = data["text"].str.replace(r"[^\w\s]","", regex=True)
data["text"] = data["text"].str.replace(r"[^\w\s]","", regex=True)
data["negativereason"] = data["negativereason"].str.replace(r"[^\w\s].", " ", regex=True)
data["airline"] = data["airline"].str.replace(r"[^\w\s]", " ", regex=True)
data["tweet_location"] = data["tweet_location"].str.replace(r"[^\w\s].", " ", regex=True)
data["user_timezone"] = data["user_timezone"].str.replace(r"[^\w\s].", " ", regex=True)

In [24]:
data.head()
#lets remove html tags URL's




,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,customer service issue,1.0000,virgin america,cairdin,0,virginamerica what dhepburn said,2015-02-24 11:35:52 -0800,los angeles,eastern time s canada)
1,570301130888122368,positive,0.3486,customer service issue,0.0000,virgin america,jnardino,0,virginamerica plus youve added commercials to ...,2015-02-24 11:15:59 -0800,los angeles,pacific time s canada)
2,570301083672813571,neutral,0.6837,customer service issue,1.0000,virgin america,yvonnalynn,0,virginamerica i didnt today must mean i need t...,2015-02-24 11:15:48 -0800,lets play,central time s canada)
3,570301031407624196,negative,1.0000,bad flight,0.7033,virgin america,jnardino,0,virginamerica its really aggressive to blast o...,2015-02-24 11:15:36 -0800,los angeles,pacific time s canada)
4,570300817074462722,negative,1.0000,can tell,1.0000,virgin america,jnardino,0,virginamerica and its a really big bad thing a...,2015-02-24 11:14:45 -0800,los angeles,pacific time s canada)


In [25]:
#data["text"][18]
data["text"][21]

'virginamerica i love this graphic '

In [26]:
#next day

In [27]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,customer service issue,1.0000,virgin america,cairdin,0,virginamerica what dhepburn said,2015-02-24 11:35:52 -0800,los angeles,eastern time s canada)
1,570301130888122368,positive,0.3486,customer service issue,0.0000,virgin america,jnardino,0,virginamerica plus youve added commercials to ...,2015-02-24 11:15:59 -0800,los angeles,pacific time s canada)
2,570301083672813571,neutral,0.6837,customer service issue,1.0000,virgin america,yvonnalynn,0,virginamerica i didnt today must mean i need t...,2015-02-24 11:15:48 -0800,lets play,central time s canada)
3,570301031407624196,negative,1.0000,bad flight,0.7033,virgin america,jnardino,0,virginamerica its really aggressive to blast o...,2015-02-24 11:15:36 -0800,los angeles,pacific time s canada)
4,570300817074462722,negative,1.0000,can tell,1.0000,virgin america,jnardino,0,virginamerica and its a really big bad thing a...,2015-02-24 11:14:45 -0800,los angeles,pacific time s canada)


In [28]:
#we wont remove stopwords because If we are training a model for senUment analysis, we might not wish to remove the stop words.
#Movie review: “The movie was not good at all.”
#Text a.er removal of stop words: “movie good”

In [30]:
#lets perform text correction
!pip install textblob      #for installation

import textblob            #to import
from textblob import TextBlob

ERROR: Invalid requirement: '#for'


Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'textblob'

In [31]:
data['text'] = data['text'].apply(lambda x: str(TextBlob(x).correct()))

NameError: name 'TextBlob' is not defined

In [32]:
#lets perfform Expanding contractions
import contractions

def expand_contractions(text):
    return contractions.fix(text)

data["text"] = data["text"].apply(expand_contractions)

In [33]:
#so far in our text column we have performed , null values replacement , dropped duplicates , convert to lowercase ,
#remove specific character ,Expanding contractions

In [34]:
from nltk import word_tokenize, sent_tokenize
data["text"]=data["text"].apply(word_tokenize)

In [35]:
data.head()
#we dont need user_timezone ,tweet_location , tweet_created,name,airline,
data.drop(['user_timezone','tweet_location' , 'tweet_created','name','airline'],axis=1,inplace=True)

In [36]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,retweet_count,text
0,570306133677760513,neutral,1.0000,customer service issue,1.0000,0,"[virginamerica, what, dhepburn, said]"
1,570301130888122368,positive,0.3486,customer service issue,0.0000,0,"[virginamerica, plus, you, have, added, commer..."
2,570301083672813571,neutral,0.6837,customer service issue,1.0000,0,"[virginamerica, i, did, not, today, must, mean..."
3,570301031407624196,negative,1.0000,bad flight,0.7033,0,"[virginamerica, its, really, aggressive, to, b..."
4,570300817074462722,negative,1.0000,can tell,1.0000,0,"[virginamerica, and, its, a, really, big, bad,..."


In [37]:
#lets perform 
data['text']=data['text'].apply(str)
import nltk
from nltk.stem import PorterStemmer
# create an object of class PorterStemmer
porter = PorterStemmer()
def stemming(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)
data['text'] = data['text'].apply(stem_sentence)


NameError: name 'stem_sentence' is not defined

In [ ]:
data.head()

In [ ]:
data.head()